# Lab 4: Analyzing Network traffic log data using Python
### Main Tasks
1. Top 5 Talkers
2. Top 5 Listeners
3. Top 5 applications
4. Traffic intensity
5. Proportion of TCP and UDP packets

### Optional Tasks
6. Top 5 communication pair 
7. Visualizing the communication between different IP hosts. 

In [1]:
# import necessary libraries 
import numpy as np
import pandas as pd
import seaborn as sb
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
# Global variable
filename = "test_Data.csv"

In [12]:
mainDF = pd.read_csv(filename,header=None) #read the csv into a pandas dataframe
flowDF = mainDF[mainDF[0] == "FLOW"] #we will weed out the CNTR entries 


print(f'Original entries \'s shape: {mainDF.shape}')
print(f'Flow only entries \'s shape: {flowDF.shape}')

Original entries 's shape: (1200, 21)
Flow only entries 's shape: (1171, 21)


In [103]:
#Let's add column headers to make things easier for us
flowDF.columns = ["FLOW","sflow_agent_address","inputPort","outputPort",
                 "src_MAC","dst_MAC","ethernet_type","in_vlan","out_vlan",
                  "src_IP","dst_IP","IP_protocol","ip_tos","ip_ttl",
                  "src_port_addr_tplayer",
                  "dst_port_addr_tplayer",
                  "tcp_flags","packet_size","IP_size","sampling_rate",
                  "Untitled"
                 ]
#flowDF.to_csv(f'Header_added_{filename}')

1. Top 5 Talkers

_For this task, we will count the src_IP that appears the most in the dataframe_

In [20]:
srcDF = flowDF.src_IP.value_counts()
srcDF.head(5)

152.3.219.19       126
207.241.228.157     66
130.14.250.13       63
193.62.192.8        46
192.122.131.36      40
Name: src_IP, dtype: int64

2. Top 5 Listeners

_Same as task 1, but dst_IP_

In [24]:
dstDF = flowDF.dst_IP.value_counts()
dstDF.head(5)

198.71.44.98      126
103.37.198.100     98
210.48.222.9       66
137.132.228.15     50
202.21.159.244     38
Name: dst_IP, dtype: int64

3. Top 5 applications

_This can be done by looking at the destination port number_

In [54]:
appDF = flowDF.dst_port_addr_tplayer.value_counts()
appDF.head(5)

443      234
56152     82
0         81
43930     45
80        37
Name: dst_port_addr_tplayer, dtype: int64

We observed that port 443 is most frequently used, which means most traffic are being passed on secured HTTP


4. Traffic intensity

_We will calculate the total number of bytes (using packet_size) sent over this duration to estimate the traffic intensity. Additionally, this is an important parameter of the network to indicate if there is congestion (simply by comparing traffic to bandwidth)_

In [104]:
packet_traffic = flowDF.packet_size.sum() #assume the unit is bytes
packet_traffic_MB = packet_traffic/1024/1024 #convert to MB
print(f'The network traffic in that period is: {packet_traffic_MB:.2f}MB')

#similarly let's see how much IP bytes are being sent
IP_traffic = flowDF.IP_size.sum()
IP_traffic_MB = IP_traffic/1024/1024 
print(f'The IP traffic in that period is: {IP_traffic_MB:.2f}MB')

MAC_header = 1-IP_traffic_MB/packet_traffic_MB
print(f'Proportion of MAC header as part of network traffic: {MAC_header*100:.2f}%')

The network traffic in that period is: 1.05MB
The IP traffic in that period is: 1.03MB
Proportion of MAC header as part of network traffic: 2.24%


5. TCP UDP percentage
<br>

_We can use the IP_protocol field to see which connection is run on tcp or udp.
As defined in the manual, tcp connection would have the value 6 and udp connection would have a value of 17_


In [33]:
tcpDF = flowDF[flowDF.IP_protocol == 6] #tcp is 6 udp is 17
tcp_count = tcpDF.shape[0]
udpDF = flowDF[flowDF.IP_protocol == 17]
udp_count = udpDF.shape[0]

total_traffic = flowDF.shape[0]

print(f'Number of tcp traffic: {tcp_count}')
print(f'Number of udp traffic: {udp_count}')
print()
print(f'Proportion tcp traffic: {tcp_count/total_traffic*100:.2f}%')
print(f'Proportion udp traffic: {udp_count/total_traffic*100:.2f}%')

Number of tcp traffic: 879
Number of udp traffic: 135

Proportion tcp traffic: 75.06%
Proportion udp traffic: 11.53%


In [51]:
#Additionally we can do this for all traffic type
IP_protocolDF = pd.DataFrame(flowDF.IP_protocol.value_counts())
total_traffic = IP_protocolDF.IP_protocol.sum()
IP_protocolDF['proportion'] = (IP_protocolDF[IP_protocolDF.columns[0]]/total_traffic)*10000//1/100
IP_protocolDF.sort_values(by = 'proportion',ascending=False)

,IP_protocol,proportion
6,879,75.06
50,140,11.95
17,135,11.52
47,14,1.19
41,2,0.17
1,1,0.08


6. Top 5 Communication Pair

_For this part we will do a multiindex dataframe using src_IP and dst_IP and then count by this index_

In [90]:
ip_pair = flowDF.groupby(['src_IP','dst_IP']).count()
ip_pair = pd.DataFrame(ip_pair.FLOW) #only take the first column 
ip_pair = ip_pair.sort_values(by='FLOW',ascending=False)
ip_pair.columns=["Count"]
ip_pair.head(5)

,,Count
src_IP,dst_IP,
152.3.219.19,198.71.44.98,126
207.241.228.157,210.48.222.9,66
130.14.250.13,103.37.198.100,63
193.62.192.8,137.132.228.15,46
130.14.250.11,103.37.198.100,35


7. Visualizing 

_We will reuse the ip_pair data frame to visualize the host-to-host communications_

In [101]:
#Recall that in Section 6 we already have a dataFrame with multiindex of src_Ip and dst_IP, let's just reuse that
ip_pair_list = list(ip_pair.index)

# building a graph based on the IP pair
G = nx.Graph()
for pair in ip_pair_list:
    G.add_edge(pair[0], pair[1])

In [105]:
# visualize the graph on networkx, but first let's ignore all the matplotlidDeprecationWarnings
import warnings
warnings.filterwarnings("ignore")

plt.figure(2,figsize=(100,100)) 
nx.draw(G, with_labels = True, font_size =10)
plt.savefig("network_by_ip.png") # save as png
plt.show()